In [ ]:
import os
import json
import glob
import warnings
import itertools
import numpy as np
import pandas as pd
import matplotlib as mpl
import ruamel.yaml as yaml
import matplotlib.pyplot as plt



from tqdm.auto import tqdm
from copy import deepcopy
from collections import OrderedDict

from multiresticodm.utils import *
from multiresticodm.config import Config
from multiresticodm.outputs import Outputs
from dasim.notebook_functions import *
from multiresticodm.sim_models import TotallyConstrained, ProductionConstrained

In [ ]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

## Import samples

In [ ]:
# Specify experiment id
experiment_id = "SIM_NN_LowNoise_test_06_08_2023_22_38_11"


In [ ]:
# Read config
conf = Config(
    path = f"../data/outputs/cambridge_work_commuter_lsoas_to_msoas/{experiment_id}/config.json"
)

In [ ]:
# Import outputs
outputs = Outputs(
    config = f"../data/outputs/cambridge_work_commuter_lsoas_to_msoas/{experiment_id}",
    output_names = [],#['alpha','beta','log_destination_attraction  ','intensity'],
    slice_samples = False,
    level = 'debug'
)

In [ ]:
vars(outputs._data).keys()

In [ ]:
output_folders = ['230806-223945_Cambridge_dataset']

In [ ]:
low_noise_data = {"TotalConstrained":{},"ProductionConstrained":{}}
high_noise_data = {"TotalConstrained":{},"ProductionConstrained":{}}

low_noise_config = {"TotalConstrained":{},"ProductionConstrained":{}}
high_noise_config = {"TotalConstrained":{},"ProductionConstrained":{}}

keys_of_interest = ["alpha","beta","kappa","loss","pred_dest_sizes"]

for output_folder in output_folders:
    # Define directory
    outputs_data_path = f'../../NeuralABM/outputs/HarrisWilson/{output_folder}/data/'
    config_path = f"../../NeuralABM/outputs/HarrisWilson/{output_folder}/config/run_cfg.yml"
    for filepath in tqdm(sorted(glob.glob(os.path.join(outputs_data_path,"*")))):
        # Read config
        with open(os.path.join(filepath,'config.yml'), "r") as stream:
            config = yaml.safe_load(stream)
        data_slice = read_h5_data(filepath,keys=keys_of_interest)
        sigma = config['HarrisWilson']['Training']['true_parameters'].get('sigma',-1)
        sim_type = config['HarrisWilson']['FlowModel'].replace('ABM','')
        if sigma <= 0.05 and sigma >= 0:
            low_noise_config[sim_type] = config
            if not str_in_list(low_noise_config[sim_type]['seed'],low_noise_data[sim_type].keys()): 
                low_noise_data[sim_type][low_noise_config[sim_type]['seed']] = dict(zip(keys_of_interest,[None]*len(keys_of_interest)))
            for k in keys_of_interest:
                if low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k] is None:
                    low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k] = data_slice[k]
                else:
                    low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k] = np.append(
                        low_noise_data[sim_type][low_noise_config[sim_type]['seed']][k],
                        data_slice[k],
                        axis=0
                    )
        elif sigma > 0.05:
            high_noise_config[sim_type] = config
            if not str_in_list(high_noise_config[sim_type]['seed'],high_noise_data[sim_type].keys()): 
                high_noise_data[sim_type][high_noise_config[sim_type]['seed']] = dict(zip(keys_of_interest,[None]*len(keys_of_interest)))
            for k in keys_of_interest:
                if high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k] is None:
                    high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k] = data_slice[k]
                else:
                    high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k] = np.append(
                        high_noise_data[sim_type][high_noise_config[sim_type]['seed']][k],
                        data_slice[k],
                        axis=0
                    )

In [ ]:
outputs._data.kappa['kappa'].values

In [ ]:
low_noise_data['TotalConstrained'][1]['kappa']

In [ ]:
plt.hist(outputs._data.alpha['alpha'].values)

In [ ]:
plt.hist(low_noise_data['TotalConstrained'][1]['alpha'])